In [ ]:
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
import requests
import torch
from PIL import Image
import json
import numpy as np
from torch.utils.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from torch.optim import AdamW
import transformers
from transformers import AutoModelForCausalLM,AutoProcessor,get_scheduler

In [ ]:
model_dtype = next(model.parameters()).dtype

In [ ]:
model_id = "microsoft/Florence-2-large-ft"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
).eval().to(device)


processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
image = Image.open("/home/jupyter/project/real_test/777.jpg")

def run_example(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input

    inputs = processor(text=prompt, images=image, return_tensors="pt")
    
    inputs = {
        k: v.to(device, dtype=model.dtype if k == "pixel_values" else None)
        for k, v in inputs.items()
    }

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )
    return parsed_answer

task_prompt = '<OCR>'
result = run_example(task_prompt)
print(result["<OCR>"])

In [ ]:

model_path = "/home/jupyter/project/modelFinal_checkpoints_epoch5"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
).eval().to(device)


processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
image = Image.open("/home/jupyter/project/real_test/777.jpg")

def run_example(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input

    inputs = processor(text=prompt, images=image, return_tensors="pt")
    
    inputs = {
        k: v.to(device, dtype=model.dtype if k == "pixel_values" else None)
        for k, v in inputs.items()
    }

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )
    return parsed_answer
task_prompt = '<OCR>'
result = run_example(task_prompt)

print(result["<OCR>"])